In [23]:
import os
from Bio import Align


In [33]:
pdb_file = './pdb/7XNO_A.pdb'
pdb_id = os.path.basename(pdb_file).split('.pdb')[0]
fasta_file = f'./fasta/{pdb_id}.fasta'

In [34]:
pdb_seqls = os.popen(f'pdb_tofasta {pdb_file}').readlines()[1:]
pdb_seq = ''.join([i.strip() for i in pdb_seqls]).replace('X', '')
print(pdb_seq)

ARSYGNGVYCNNKKCWVNRGEATQSIIGGMISGWASGLAGM


In [35]:
with open(fasta_file, 'r') as f:
    fasta_seq = f.readlines()[1].strip()
print(fasta_seq)

ARSYGNGVYCNNKKCWVNRGEATQSIIGGMISGWASGLAGM


In [32]:
aligner = Align.PairwiseAligner()
alignment = aligner.align(fasta_seq, pdb_seq)[0]
print(alignment)
assert len(alignment[0]) == len(fasta_seq)

target            0 ARSYGNGVYCNNKKCWVNRGEATQSIIGGMISGWASGLAGM 41
                  0 ||||||||||||||||||||||||||||||||||||||||| 41
query             0 ARSYGNGVYCNNKKCWVNRGEATQSIIGGMISGWASGLAGM 41



ATOM      1  N   ALA A   1     119.914 153.030 113.152  1.00 63.64      A    N  



In [37]:
p_pdb = 0
p_fst = 0
mapping = {}
for i in alignment[1]:
    p_fst += 1
    if i != '-':
        p_pdb += 1
        mapping[p_pdb] = p_fst

print(mapping)


{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41}


In [38]:
def remap_resnum_for_line(pdb_file, mapping):
    newlines = []
    last_resstr = ""
    resi = 0

    with open(pdb_file, 'r') as f:
        lines = f.readlines()
    for line in lines:
        if not line.startswith('ATOM '):
                continue
        left = line[:22]
        right = line[27:]
        resstr = line[22:27]
        if resstr == last_resstr:
            newstr = "%4d " % mapping[resi]
        else:
            last_resstr = resstr
            if resi != len(mapping):
                resi += 1
            if resi == len(mapping):
                resi = resi
            newstr = "%4d " % mapping[resi]
        #print("|"+resstr+"|"+newstr+"|")
        newlines.append(left+newstr+right)
    return newlines

In [42]:
new_lines = remap_resnum_for_line(pdb_file, mapping)
with open(f'./pdb_renum/{pdb_id}_renum.pdb', 'w') as f:
    f.writelines(new_lines)


In [40]:
new_lines[1]

'ATOM      2  CA  ALA A   1     118.819 152.688 114.052  1.00 63.64      A    C  \n'